In [1]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os
import talib
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import requests
import pandas as pd
import datetime
import warnings
import sys


In [2]:
def get_allstock_info():
    # 獲取所有股票的信息，包括上市和上櫃股票
    url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
    response = requests.get(url)
    listed = pd.read_html(response.text)[0]
    listed.columns = listed.iloc[0, :]
    listed = listed[["有價證券代號", "有價證券名稱", "市場別", "產業別", "公開發行/上市(櫃)/發行日"]]
    listed = listed.iloc[1:]

    # 上櫃股票
    urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
    response = requests.get(urlTWO)
    listedTWO = pd.read_html(response.text)[0]
    listedTWO.columns = listedTWO.iloc[0, :]
    listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
    listedTWO = listedTWO[["有價證券代號", "有價證券名稱", "市場別", "產業別", "公開發行/上市(櫃)/發行日"]]

    # 上市和上櫃股票代號處理
    stock_num = pd.concat([listed["有價證券代號"].apply(lambda x: str(x) + ".TW"),
                           listedTWO["有價證券代號"].apply(lambda x: str(x) + ".TWO")], ignore_index=True)
    stock_num.loc[len(stock_num) + 1] = '0050.TW'
    stock_num.loc[len(stock_num) + 1] = '^TWII'
    stock_num = stock_num.sort_values(ascending=False).reset_index(drop=True)

    # 合併所有股票信息
    allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
    allstock_info.columns = ["ID", "有價證券名稱", "市場別", "產業別", "公開發行/上市(櫃)/發行日"]
    allstock_info.set_index('ID', inplace=True)
    return stock_num, allstock_info

In [3]:
stock_num, allstock_info = get_allstock_info()

In [4]:
BASE_DIR = r'c:\Users\User\Desktop\stock'
file_dir = os.path.join(BASE_DIR, 'initialize_dataset')
data_dir = os.path.join(BASE_DIR, 'data_test')
print(file_dir)
print(data_dir)

c:\Users\User\Desktop\stock\initialize_dataset
c:\Users\User\Desktop\stock\data_test


In [ ]:
## 第一次製做獨立的RS rank csv
def rs_rate():
    files = glob.glob(os.path.join(data_dir, '*.csv'))
    print(files[:5])
    rs_rank_list = []
    rank_columns = ["RS 250MA", "RS 50MA", "RS 20MA", "RS 250EMA", "RS 50EMA", "RS 20EMA", "Date", "ID"]
    for i, path in enumerate(files):    
        try:
            hisdata = pd.read_csv(path,parse_dates = ["Date"])
            rs = hisdata[rank_columns]
            if i == 0:
                # rs_rank_df = rs[rank_columns]
                rs_rank_list.append(rs[rank_columns])
            else:
                # rs_rank_df = pd.concat([rs_rank_df, rs[rank_columns]], ignore_index=True)
                rs_rank_list.append(rs[rank_columns])
        except Exception as e:
            print(e)
        print(f'{path.split("/")[-1]} done.  {i+1}/{len(files)}')
    rs_rank_df = pd.concat(rs_rank_list, ignore_index=True)
    rs_rank_df.to_csv(f'{file_dir}/historical_rs_value_MA_EMA.csv')
rs_rate()

In [15]:
## 第一次建立RS rank的csv
files_1 = pd.read_csv(f'{file_dir}/historical_rs_value_MA_EMA.csv', parse_dates=['Date'])

In [ ]:
files = files_1.copy()
dates = list(set(files['Date'].values))
files['Date'] = pd.to_datetime(files['Date'], utc=True)
files.set_index('Date', inplace=True)
files.dropna(inplace=True)
# files['RS_rate_20'] = 0
# files['RS_rate_50'] = 0
# files['RS_rate_250'] = 0
# files['ERS_rate_20'] = 0
# files['ERS_rate_50'] = 0
files['ERS_rate_250'] = 0
MA_list = ['20', '50', '250']
MA_type_list = ['', 'E']
scaler = MinMaxScaler(feature_range=(0, 100))
length = len(dates)
# files.drop(files[files['ID']=='^TWII'].index, inplace=True)



for MA in MA_list:
    for MA_type in MA_type_list:
        rank_column = f'{MA_type}RS_rank_{MA}'
        rate_column = f'{MA_type}RS_rate_{MA}'
        
        # try:
        # 計算 rank
        files[rank_column] = files.groupby('Date')[f'RS {MA}{MA_type}MA'].rank(method='dense', ascending=True)

        # 縮放處理 (批次處理)
        scaler = MinMaxScaler(feature_range=(0, 100))
        files[rate_column] = files.groupby('Date')[rank_column].transform(
            lambda x: scaler.fit_transform(x.values.reshape(-1, 1)).flatten()
        )

        # 刪除不必要的欄位
        files.drop(rank_column, axis=1, inplace=True)
            
        # except Exception as e:
        #     print(f'計算 {rate_column} 時發生錯誤: {e}')

files.to_csv(f'{file_dir}/rs_EMA_MA_rank_and_rate.csv')

In [24]:
## 第一次匯入所有歷史RS rank
address_1 = pd.read_csv(f'{file_dir}/rs_EMA_MA_rank_and_rate.csv',parse_dates = ["Date"])

In [34]:
address = address_1.copy()
address['Date'] = address['Date'].dt.strftime('%Y-%m-%d')
address.set_index('Date', inplace=True)

In [38]:
stock_length = len(stock_num)
rs_rate_columns = ['RS_rate_20', 'RS_rate_50', 'RS_rate_250', 'ERS_rate_20', 'ERS_rate_50', 'ERS_rate_250']
for n, i in enumerate(stock_num):
# for n, i in enumerate(['0050.TW', '1101.TW', '1102.TW', '1103.TW', '1104.TW']):
    print(f'{n}/{stock_length}  {i}', end=' | ')
    try:
        stock = pd.read_csv(f'{data_dir}/' + str(i) + '.csv',parse_dates = ["Date"]).set_index('Date')
        stock_historical_data = address.loc[address['ID'] == i]

        # 保存原始的 Date 格式
        original_index = stock.index

        # 臨時統一日期格式
        stock.index = pd.to_datetime(stock.index).tz_localize(None)
        stock_historical_data.index = pd.to_datetime(stock_historical_data.index).tz_localize(None)

        #  確保有有效日期
        valid_dates = stock.index.intersection(stock_historical_data.index)

        if valid_dates.empty:
            print(f'無有效日期匹配，跳過 {i}')
            continue
        # print(stock.loc[valid_dates[-5:], 'Close'])
        # print(stock_historical_data.loc[valid_dates[-5:], rs_rate_columns])
        # 賦值操作
        stock.loc[valid_dates, rs_rate_columns] = stock_historical_data.loc[valid_dates, rs_rate_columns]

        # print(stock.loc[valid_dates[-5:], rs_rate_columns])
        # print(stock.loc[valid_dates[-5:], 'Close'])
        # print(stock.loc[valid_dates[-5:]])

        # 恢復原始 Date 格式
        stock.index = original_index
        stock.to_csv(f'{data_dir}/' + str(i) + '.csv', index=True, encoding='utf-8-sig')
        print('rs_rank 已完成', end=' | ')
    except Exception as e:
        print(e)
        print('rs_rank 更新失敗', end=' | ')
    print('')

0/1841  ^TWII | rs_rank 已完成 | 
1/1841  9962.TWO | rs_rank 已完成 | 
2/1841  9960.TWO | rs_rank 已完成 | 
3/1841  9958.TW | rs_rank 已完成 | 
4/1841  9955.TW | rs_rank 已完成 | 
5/1841  9951.TWO | rs_rank 已完成 | 
6/1841  9950.TWO | rs_rank 已完成 | 
7/1841  9949.TWO | rs_rank 已完成 | 
8/1841  9946.TW | rs_rank 已完成 | 
9/1841  9945.TW | rs_rank 已完成 | 
10/1841  9944.TW | rs_rank 已完成 | 
11/1841  9943.TW | rs_rank 已完成 | 
12/1841  9942.TW | rs_rank 已完成 | 
13/1841  9941.TW | rs_rank 已完成 | 
14/1841  9940.TW | rs_rank 已完成 | 
15/1841  9939.TW | rs_rank 已完成 | 
16/1841  9938.TW | rs_rank 已完成 | 
17/1841  9937.TW | rs_rank 已完成 | 
18/1841  9935.TW | rs_rank 已完成 | 
19/1841  9934.TW | rs_rank 已完成 | 
20/1841  9933.TW | rs_rank 已完成 | 
21/1841  9931.TW | rs_rank 已完成 | 
22/1841  9930.TW | rs_rank 已完成 | 
23/1841  9929.TW | rs_rank 已完成 | 
24/1841  9928.TW | rs_rank 已完成 | 
25/1841  9927.TW | rs_rank 已完成 | 
26/1841  9926.TW | rs_rank 已完成 | 
27/1841  9925.TW | rs_rank 已完成 | 
28/1841  9924.TW | rs_rank 已完成 | 
29/1841  9921.TW | rs